In [1]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
from tqdm import trange
import pyscipopt as pyscip
import pickle as pkl
from dump_data import *

scip_parameters = {"branching/scorefunc": "s",
                   "limits/time": 360,
                   "conflict/enable": False,
                   
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   
                   "separating/maxrounds": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   
                   "propagating/maxroundsroot" : 0,
                   "propagating/maxrounds" : 0,
                   
                   "randomization/permutevars": False,
                   "randomization/permuteconss": False,
                   "randomization/permutationseed": 0,
                  }
env = ecole.environment.Branching(observation_function = ecole.observation.NodeBipartite(),\
                  scip_params = scip_parameters)


In [2]:
done = 0
while not done:
    try:
        setCover = ecole.instance.SetCoverGenerator\
                    (n_rows=5, n_cols=5,density=0.8)
        instance = next(setCover)
        instance.write_problem("debugFile/unitest/pb.lp")

        obs, action_set, _, _, _ = env.reset(instance)
        print(obs.row_features.shape)
        print(obs.edge_features.shape)
        print(obs.variable_features.shape)
        done = 1
    except Exception as ex:
#         print("Erreur:%s"%ex)
        
        pass

(5, 5)
[5, 5]
(5, 20)


In [3]:
done = 0
while not done:
    try:
        instance = ecole.scip.Model.from_file("debugFile/unitest/pb.lp")
        obs, action_set, _, _, _ = env.reset(instance)
        print(obs.row_features.shape)
        print(obs.edge_features.shape)
        print(obs.variable_features)
        done = 1
    except Exception as ex:
#         print("Erreur:%s"%ex)
        pass
    
dumpRowFeatures("debugFile/unitest/constraints_features.json",obs.row_features)
#save variables features
dumpVariableFeatures("debugFile/unitest/variables_features.json",obs.variable_features)
#save edges features
original_indice = obs.variable_features[:,-1]
dumpEdgeFeatures("debugFile/unitest/edges_features.json",obs.edge_features,original_indice)
#get et save label
aspyscip = instance.as_pyscipopt()
aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
aspyscip.optimize()
dumpSolution_Ecole("debugFile/unitest/label.json",aspyscip)

(5, 5)
[5, 5]
[[ 0.43803848  1.          0.          0.          0.          1.
   1.          0.2416764  -0.         -0.          1.          0.
   0.11111111  0.          0.07792208  1.          0.          0.
   0.          1.        ]
 [ 0.77034353  1.          0.          0.          0.          1.
   1.          0.57398146 -0.         -0.          1.          0.
   0.11111111  0.          0.02597403  1.          0.          0.
   0.          2.        ]
 [ 0.21146685  1.          0.          0.          0.          1.
   1.         -0.          0.5         0.5         0.          0.
   0.          1.          0.54545455  0.          1.          0.
   0.          0.        ]
 [ 0.19636208  1.          0.          0.          0.          1.
   1.         -0.          0.5         0.5         0.          0.
   0.          1.          0.70562771  0.          1.          0.
   0.          3.        ]
 [ 0.3625146   1.          0.          0.          0.          1.
   1.         -0.   

In [4]:
#read features of variables 
import csv
import json
import numpy as np

var_file = open("debugFile/unitest/variables_features.json","rb")
data = json.load(var_file)
data_var = np.array(data["values"])
data_name = np.array(data["names"])

csvfile = open("debugFile/unitest/Feature.csv","wt",newline="") 
writer = csv.writer(csvfile,delimiter = ",")
writer.writerows(zip(data_name.tolist(),data_var[0].tolist(),data_var[1].tolist(),data_var[2].tolist(),data_var[3].tolist(),data_var[4].tolist()))

# for j in range(len(data_var)):
#     for i in range(len(data_name)):
#         print(data_name[i],data_var[j][i])
#     print()
var_file = open("debugFile/unitest/constraints_features.json","rb")
data = json.load(var_file)
data_var = np.array(data["values"])
data_name = np.array(data["names"])

writer.writerows(" ")
writer.writerows(zip(data_name.tolist(),data_var[0].tolist(),data_var[1].tolist(),data_var[2].tolist(),data_var[3].tolist(),data_var[4].tolist()))


csvfile.close()